In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import urllib
import copy
from copy import deepcopy
import matplotlib.pyplot as plt

# Austria Data Input

In [23]:
austria = pd.read_csv('Data/austria.csv')
austria_shp = gp.read_file('Data/austria.shp')

aust = austria[austria['Origin'] != austria['Destination']] #intra-zonal ﬂows have been excluded
aust_Y = aust['Data'].values #the number of ﬂows between i and j
Oi = aust['Oi'].values 
Dj = aust['Dj'].values 
dij = aust['Dij'].values 
w = np.ones(72).reshape((-1,1))
aust_X = np.column_stack([Oi, dij,Dj])

In [6]:
#sort shapefile and dataframe so that places will be in the same order(ascending in this case)
aust_shp = austria_shp.copy()
aust_shp = aust_shp.sort_values(by = 'NUTS_ID') # change the order of places in aust_shp file as descending
Dja = austria.drop_duplicates(subset=['Destination'])
Dja = Dja.sort_values(by = 'Destination')['Dj'].values 

In [7]:
from mgwr.sel_bw import Sel_BW
from mgwr.gwr import GWR, MGWR
import pandas as pd
import libpysal as ps

In [8]:
points = aust_shp.copy()
points = points.to_crs("EPSG:3416")
points.geometry = points.geometry.centroid

In [9]:
points['X'] = points.geometry.x
points['Y'] = points.geometry.y

In [11]:
points.head()

,OBJECTID,NUTS_ID,STAT_LEVL_,AREA,LEN,Shape_Leng,Shape_Area,geometry,X,Y
2,316,AT11,2,0.0,0.0,7.250182,0.474910,POINT (639794.320 409576.733),639794.319818,409576.733087
7,337,AT12,2,0.0,0.0,12.106180,2.326684,POINT (579496.916 488557.414),579496.915685,488557.413879
3,326,AT13,2,0.0,0.0,1.429078,0.050059,POINT (626152.234 483188.811),626152.234389,483188.810852
5,334,AT21,2,0.0,0.0,7.266858,1.122715,POINT (444692.586 319642.620),444692.586038,319642.619648
8,350,AT22,2,0.0,0.0,9.328439,1.951187,POINT (527128.581 375752.960),527128.581277,375752.960185


In [12]:
aust_points = aust.merge(points, left_on='Destination', right_on='NUTS_ID')

In [39]:
coords = list(zip(aust_points.X, aust_points.Y))

In [40]:
X = (aust_X - aust_X.mean(axis=0)) / aust_X.std(axis=0)
#[Oi, dij, Dj]
y = (aust_Y - aust_Y.mean(axis=0)) / aust_Y.std(axis=0)


In [41]:
selector = Sel_BW(coords, y, X, multi=True, constant=True)
bw1 = selector.search()

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

In [42]:
bw1

array([65., 65., 58., 49.])

In [43]:
bw2 = selector.search(multi_bw_min=[np.inf, np.inf, np.inf, None])

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

OverflowError: cannot convert float infinity to integer